In [2]:
%matplotlib inline


Autograd: 自动求导
===================================

PyTorch的核心是``autograd``包。
我们首先简单的了解一些，然后用PyTorch开始训练第一个神经网络。



``autograd``为所有用于Tensor的operation提供自动求导的功能。

我们通过一些简单的例子来学习它基本用法。

Tensor
--------

``torch.Tensor`` 是这个包的核心类。如果它的属性
``.requires_grad``是``True``，那么PyTorch就会追踪所有与之相关的operation。当完成(正向)计算之后，
我们可以调用``.backward()``，PyTorch会自动的把所有的梯度都计算好。与这个tensor相关的梯度都会累加到它的
``.grad``属性里。

如果不想计算这个tensor的梯度，我们可以调用``.detach()``，这样它就不会参与梯度的计算了。


为了阻止PyTorch记录用于梯度计算相关的信息(从而节约内存)，我们可以使用 ``with torch.no_grad():``。
这在模型的预测时非常有用，因为预测的时候我们不需要计算梯度，否则我们就得一个个的修改Tensor的`requires_grad`属性，
这会非常麻烦。

关于autograd的实现还有一个很重要的类``Function``.

``Tensor``和``Function``相互连接从而形成一个有向无环图。, 这个图记录了计算的完整历史。
每个tensor有一个``.grad_fn``属性来引用创建这个tensor的``Function`` that has created
the ``Tensor`` (除了用户直接创建的Tensor，这些Tensor的``grad_fn``是None)。

如果你想计算梯度，可以对一个``Tensor``调用它的``.backward()``方法。如果这个Tensor是一个scalar(只有一个数)，
那么调用时不需要传任何参数。如果Tensor多于一个数，那么需要传入和它的shape一样的参数，表示反向传播过来的梯度。




In [3]:
import torch

创建tensor时设置属性requires_grad=True，PyTorch就会记录用于反向梯度计算的信息。


In [4]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


通过operation产生新的tensor：



In [5]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward>)


``y``是通过operation产生的tensor，因此它的``grad_fn``不是None。



In [6]:
print(y.grad_fn)

再通过y得到z和out



In [7]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward>) tensor(27., grad_fn=<MeanBackward1>)


``.requires_grad_( ... )``函数会修改一个Tensor的``requires_grad``。


In [8]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


梯度
---------
现在我们里反向计算梯度。
因为``out``是一个scalar，因此``out.backward()``等价于``out.backward(torch.tensor(1))``。


In [9]:
out.backward()

打印梯度d(out)/dx




In [10]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


我们手动计算来验证一下。为了简单，我们把out记为o。
$o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ 并且 $z_i\bigr\rvert_{x_i=1} = 27$.
因此，
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$，因此
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.



我们也可以用autograd做一些很奇怪的事情！比如y和x的关系是while循环的关系(似乎很难用一个函数直接表示y和x的关系？对x不断平方直到超过1000，这是什么函数？)


In [11]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([ 267.9040, 1054.1506,  607.1303], grad_fn=<MulBackward>)


In [12]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(gradients)

print(x.grad)

tensor([ 51.2000, 512.0000,   0.0512])


我们可以使用``with torch.no_grad():``来停止梯度的计算：



In [13]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
	print((x ** 2).requires_grad)

True
True
False


**深入阅读：**

``autograd``和``Function``的文档在 http://pytorch.org/docs/autograd

